In [366]:
import gym
import torch
import torchvision
import random

env_name = 'CartPole-v0'
env = gym.make(env_name)
state = env.reset()
print(type(state))
print(type(state[0]))

print(type(torch.rand((2,2))))
print(type(torch.from_numpy(state)), torch.from_numpy(state))

<class 'numpy.ndarray'>
<class 'numpy.float64'>
<class 'torch.Tensor'>
<class 'torch.Tensor'> tensor([-0.0481,  0.0423,  0.0096, -0.0104], dtype=torch.float64)


In [367]:
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
state_dim = env.observation_space.shape
action_size = env.action_space.n
state_tensor = torch.from_numpy(state)
print(state_tensor, type(state_tensor))

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np


class QNetwork(nn.Module):
    def __init__(self, state_dim, action_size):
        super().__init__()
        print("HM")
        
        self.action_in = 0
        self.action_one_hot = F.one_hot(torch.arange(0,2)[self.action_in], num_classes=action_size).float()
        self.hidden1 = nn.Linear(*state_dim, 100) # Fully connected
        self.q_state = nn.Linear(100, action_size)
        
        self.optimizer = optim.Adam(self.parameters(), lr=0.001)

    def forward(self, x):
        x = F.relu(self.hidden1(x))
        x = self.q_state(x)
        return x

    def update_model(self, state, action, q_target):
        
        state_tensor_in = torch.from_numpy(state)
        self.action_in = action
        self.action_one_hot = F.one_hot(torch.arange(0,2)[self.action_in], num_classes=action_size)
        
        action_one_hot = F.one_hot(torch.arange(0,2)[action], num_classes=action_size).float()
        self.q_target_tensor_in = q_target
        q_state = self(state_tensor_in.float())
        q_state_action = torch.dot(q_state, action_one_hot).sum()
        
        EPOCHS = 1
        
        for epoch in range(EPOCHS):
            self.zero_grad()
            loss = F.mse_loss(q_state_action, q_target)
            loss.backward()
            self.optimizer.step()
            print("LOSS:", loss)
        
    def get_q_state(self, state):
        state_tensor = torch.from_numpy(state)
        q_state = self(state_tensor.float())
        return q_state
        

Observation space: Box(4,)
Action space: Discrete(2)
tensor([-0.0481,  0.0423,  0.0096, -0.0104], dtype=torch.float64) <class 'torch.Tensor'>


In [368]:
class DQNAgent():
    def __init__(self, env):
        self.state_dim = env.observation_space.shape
        self.action_size = env.action_space.n
        self.q_network = QNetwork(self.state_dim, self.action_size)
        self.gamma = 0.97
        self.eps = 1.0
        
    def get_action(self, state):
        q_state = self.q_network.get_q_state(state)
        action_value, action_index = torch.max(q_state, 0)
        action_greedy = action_index.item()
        action_random = np.random.randint(self.action_size)
        action = action_random if random.random() < self.eps else action_greedy
        return action
    
    def train(self, state, action, next_state, reward, done):
        q_next_state = self.q_network.get_q_state(next_state)
        q_next_state = (1-done) * q_next_state
        q_target = reward + self.gamma * q_next_state
        self.q_network.update_model(state, action, q_target)
        
        if done:
            self.eps = 0.99 * self.eps
        

In [369]:
    agent = DQNAgent(env)
    num_episodes = 50

    for ep in range(num_episodes):
        state = env.reset()
        total_reward = 0
        done = False
        while not done:
            action = agent.get_action(state)
            next_state, reward, done, info = env.step(action)
            agent.train(state, action, next_state, reward, done)
            env.render()
            total_reward += reward
            state = next_state
        print("Episode: {}, total_reward: {:2f}".format(ep, total_reward))


HM
LOSS: tensor(0.7807, grad_fn=<MeanBackward0>)
LOSS: tensor(1.3897, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2567, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8731, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2289, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8847, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7886, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7228, grad_fn=<MeanBackward0>)
LOSS: tensor(1.4226, grad_fn=<MeanBackward0>)
LOSS: tensor(1.3097, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2067, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8846, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7989, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7501, grad_fn=<MeanBackward0>)


/home/jith/anaconda3/envs/pytorch-env/lib/python3.7/site-packages/ipykernel_launcher.py:46: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


LOSS: tensor(1.3373, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2557, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1844, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0672, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9811, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8972, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1502, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9010, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8345, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1816, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1345, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0449, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9961, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9424, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0718, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0536, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9562, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9176, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8961, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8813, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1197, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8423, grad_fn=<Mean

LOSS: tensor(0.9788, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0236, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9430, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0184, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0027, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9812, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9928, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9647, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9215, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0852, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0252, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9471, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9210, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0835, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0323, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9342, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0333, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9293, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0348, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9241, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0375, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9182, grad_fn=<Mean

LOSS: tensor(0.9701, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8690, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9466, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8899, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9173, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9018, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9125, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0308, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1628, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7204, grad_fn=<MeanBackward0>)
LOSS: tensor(0.3676, grad_fn=<MeanBackward0>)
Episode: 10, total_reward: 78.000000
LOSS: tensor(1.0840, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1625, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7713, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8145, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1006, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8108, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8368, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8312, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1135, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8244, grad_fn=<MeanBackward0

LOSS: tensor(0.7986, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8793, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9722, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8465, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8429, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9237, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9690, grad_fn=<MeanBackward0>)
LOSS: tensor(0.0972, grad_fn=<MeanBackward0>)
Episode: 17, total_reward: 57.000000
LOSS: tensor(1.1013, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7894, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8011, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7811, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1896, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7581, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7654, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2189, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7519, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2412, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7479, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2407, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7580, grad_fn=<MeanBackward0

LOSS: tensor(0.7158, grad_fn=<MeanBackward0>)
LOSS: tensor(0.1224, grad_fn=<MeanBackward0>)
Episode: 27, total_reward: 30.000000
LOSS: tensor(0.7806, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1639, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7897, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7749, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7777, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1989, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1880, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1306, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8163, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7955, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8148, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8520, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0717, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1060, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0935, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0388, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8942, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8886, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9419, grad_fn=<MeanBackward0

LOSS: tensor(0.0531, grad_fn=<MeanBackward0>)
Episode: 35, total_reward: 18.000000
LOSS: tensor(0.7333, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6782, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6502, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6203, grad_fn=<MeanBackward0>)
LOSS: tensor(1.4168, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6182, grad_fn=<MeanBackward0>)
LOSS: tensor(1.4091, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6251, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6238, grad_fn=<MeanBackward0>)
LOSS: tensor(0.1892, grad_fn=<MeanBackward0>)
Episode: 36, total_reward: 10.000000
LOSS: tensor(0.6528, grad_fn=<MeanBackward0>)
LOSS: tensor(0.5799, grad_fn=<MeanBackward0>)
LOSS: tensor(1.6324, grad_fn=<MeanBackward0>)
LOSS: tensor(0.5800, grad_fn=<MeanBackward0>)
LOSS: tensor(1.6191, grad_fn=<MeanBackward0>)
LOSS: tensor(0.5853, grad_fn=<MeanBackward0>)
LOSS: tensor(0.5674, grad_fn=<MeanBackward0>)
LOSS: tensor(1.6003, grad_fn=<MeanBackward0>)
LOSS: tensor(1.5371, grad_fn=<MeanBackward0>)
LOSS: 

LOSS: tensor(0.8900, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8815, grad_fn=<MeanBackward0>)
LOSS: tensor(1.0057, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7948, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9925, grad_fn=<MeanBackward0>)
LOSS: tensor(0.8048, grad_fn=<MeanBackward0>)
LOSS: tensor(0.9305, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2198, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6395, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2593, grad_fn=<MeanBackward0>)
LOSS: tensor(1.6387, grad_fn=<MeanBackward0>)
LOSS: tensor(1.9953, grad_fn=<MeanBackward0>)
LOSS: tensor(0.0042, grad_fn=<MeanBackward0>)
Episode: 44, total_reward: 25.000000
LOSS: tensor(1.1915, grad_fn=<MeanBackward0>)
LOSS: tensor(1.1674, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2114, grad_fn=<MeanBackward0>)
LOSS: tensor(0.6860, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7259, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2054, grad_fn=<MeanBackward0>)
LOSS: tensor(0.7138, grad_fn=<MeanBackward0>)
LOSS: tensor(1.2278, grad_fn=<MeanBackward0